In [2]:
!pip install py7zr

from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/temp/')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 24.1 MB/s eta 0:00:00


In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device(type='cuda', index=0)
else:
    device = torch.device(type='cpu', index=0)

In [4]:
train_labels = pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv", header = 'infer')

classes = train_labels['label'].unique()

name2num = {}
num2name = {}

i=0

for name in classes:
    name2num[name] = i
    i = i + 1

for i in range(len(classes)):
    num2name[i] = classes[i]

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os

from torchvision.io import read_image
from torchvision.transforms import Normalize, Resize, Compose


class TrainDataset(Dataset):
    def __init__(self,imgpath, labelpath):
        super().__init__()
        self.imgpath = imgpath
        self.labelpath = labelpath
        self.labels = pd.read_csv(labelpath, header = 'infer')
        self.transform = Compose([Resize((224,224), antialias = True), Normalize(mean = [0.485,0.456,0.406], std = [0.229,0.224,0.225])])


    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self,idx):
        finalpath = os.path.join(self.imgpath,str(idx+1)) + '.png'
        img = read_image(finalpath)/255
        img = self.transform(img)
        label = self.labels.iloc[idx,1]
        label = name2num[label]

        return img, label

In [6]:
traindataset = TrainDataset('/kaggle/temp/train','/kaggle/input/cifar-10/trainLabels.csv')

batch_size = 64
traindataloader = DataLoader(dataset = traindataset, batch_size = batch_size)


In [7]:
import torch.nn as nn

from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

class Cifar10(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrainednet = mobilenet_v3_large(weights = MobileNet_V3_Large_Weights.DEFAULT)
        self.pretrainednet.classifier = nn.Sequential(
            nn.Linear(in_features = 960, out_features = 1280, bias = True),
            nn.Hardswish(),
            nn.Dropout(p = 0.2, inplace = True),
            nn.Linear(in_features = 1280, out_features = 10, bias = True)
        )

    def forward(self, x):
        output = self.pretrainednet(x)

        return output

In [8]:
def train_one_epoch(dataloader, model,loss_fn, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    num_param=0
    
    for i, (imgs, labels) in enumerate(dataloader):
        imgs=imgs.to(device)
        labels=labels.to(device)
        pred=model(imgs)
                    
        loss=loss_fn(pred,labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()
        
        running_loss=round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc=round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
            
    epoch_loss=running_loss
    epoch_acc=running_acc
    return epoch_loss, epoch_acc

In [9]:
model = Cifar10().to(device)

for param in model.pretrainednet.features.parameters():
    param.requires_grad = False


loss_fn = nn.CrossEntropyLoss()

learming_rate = 0.001

optimizer = torch.optim.Adam(params = model.parameters(), lr = learming_rate)

n_epochs = 15


for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(traindataloader,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 210MB/s]


Epoch No: 1
Batch: 1 / 782 Running Loss: 2.3 Running Accuracy: 7.81
Batch: 101 / 782 Running Loss: 1.06 Running Accuracy: 63.95
Batch: 201 / 782 Running Loss: 0.92 Running Accuracy: 67.96
Batch: 301 / 782 Running Loss: 0.87 Running Accuracy: 69.74
Batch: 401 / 782 Running Loss: 0.83 Running Accuracy: 71.04
Batch: 501 / 782 Running Loss: 0.81 Running Accuracy: 71.62
Batch: 601 / 782 Running Loss: 0.8 Running Accuracy: 72.06
Batch: 701 / 782 Running Loss: 0.79 Running Accuracy: 72.46
Training: Epoch Loss: 0.78 Epoch Accuracy: 72.88
--------------------------------------------------
Epoch No: 2
Batch: 1 / 782 Running Loss: 0.62 Running Accuracy: 76.56
Batch: 101 / 782 Running Loss: 0.67 Running Accuracy: 76.67
Batch: 201 / 782 Running Loss: 0.64 Running Accuracy: 77.15
Batch: 301 / 782 Running Loss: 0.64 Running Accuracy: 77.58
Batch: 401 / 782 Running Loss: 0.64 Running Accuracy: 77.62
Batch: 501 / 782 Running Loss: 0.64 Running Accuracy: 77.62
Batch: 601 / 782 Running Loss: 0.64 Running

In [10]:
for param in model.pretrainednet.features.parameters():
    param.requires_grad=True

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(traindataloader,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

Epoch No: 1
Batch: 1 / 782 Running Loss: 0.08 Running Accuracy: 96.88
Batch: 101 / 782 Running Loss: 0.98 Running Accuracy: 75.99
Batch: 201 / 782 Running Loss: 0.79 Running Accuracy: 79.24
Batch: 301 / 782 Running Loss: 0.69 Running Accuracy: 80.97
Batch: 401 / 782 Running Loss: 0.63 Running Accuracy: 82.29
Batch: 501 / 782 Running Loss: 0.59 Running Accuracy: 83.26
Batch: 601 / 782 Running Loss: 0.55 Running Accuracy: 84.07
Batch: 701 / 782 Running Loss: 0.52 Running Accuracy: 84.76
Training: Epoch Loss: 0.5 Epoch Accuracy: 85.24
--------------------------------------------------
Epoch No: 2
Batch: 1 / 782 Running Loss: 0.2 Running Accuracy: 90.62
Batch: 101 / 782 Running Loss: 0.26 Running Accuracy: 91.51
Batch: 201 / 782 Running Loss: 0.26 Running Accuracy: 91.53
Batch: 301 / 782 Running Loss: 0.25 Running Accuracy: 91.89
Batch: 401 / 782 Running Loss: 0.24 Running Accuracy: 92.16
Batch: 501 / 782 Running Loss: 0.23 Running Accuracy: 92.4
Batch: 601 / 782 Running Loss: 0.23 Running

In [11]:
#unpacking test images, there are 3 lacs images. This will take some time
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/temp/')

#unregister unpack format, we are done with it
shutil.unregister_unpack_format('7zip')#, ['.7z'], unpack_7zarchive)

In [12]:
class TestDataset(Dataset):
    def __init__(self, imgpath):
        super().__init__()
        self.imgpath=imgpath
        _,_,self.files=next(os.walk(self.imgpath))
        self.length=len(self.files)
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])        
    
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'
        img=read_image(finalpath)/255.0
        img=self.transform(img)
        return img

testdataset=TestDataset('/kaggle/temp/test/')
testdataloader=DataLoader(dataset=testdataset, batch_size=batch_size)

In [13]:
def eval(dataloader, model,loss_fn, path):
    model.eval()
    data=pd.read_csv(path)
    with torch.no_grad():
        for i, imgs in enumerate(dataloader):
            finalbatchpred=np.zeros(imgs.shape[0],dtype='object')
            imgs=imgs.to(device)
            pred=model(imgs)
            
            pred=torch.argmax(pred,dim=1).type(torch.int).cpu()
            for j,p in enumerate(pred):
                finalbatchpred[j]=num2name[p.item()]
            data.iloc[i*batch_size:i*batch_size+batch_size ,1]=finalbatchpred
    
    data.to_csv('submission.csv', index=False)
    data.head()

In [14]:
eval(testdataloader, model,loss_fn, '/kaggle/input/cifar-10/sampleSubmission.csv')